### Load and evaluate model on validation and test dataset

In [1]:
import sys
sys.path.append("./src") # append to system path

from sklearn import cross_validation
from sklearn.metrics import r2_score
from sklearn.externals import joblib

import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

/home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
# Helpers
def mre(true_y,pred_y):
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.abs(np.subtract(true_y, pred_y) / true_y) * 100

def run_model(graph_path, model_path, inputs):
    # load graph and model 
    # return prediction on the inputs
    with tf.Session() as sess:
        new_saver = tf.train.import_meta_graph(graph_path)
        new_saver.restore(sess, model_path)
        X = tf.get_collection("X")[0]
        y = tf.get_collection("y")[0]
        pred = tf.get_collection("pred")[0]  
        results = sess.run(pred,feed_dict={X:inputs})
        return results

### Run prediction on validation and set

In [3]:
trn_descs = pd.read_csv('../data/descs/train/descs_Mar08_3839_train.csv',header=0,index_col=None).fillna(0).values
tst_descs = pd.read_csv('../data/descs/test/descs_Mar08_3839_test.csv',header=0,index_col=None).fillna(0).values

In [4]:
### Change here
RS = 42
# load target
trn_target = pd.read_csv('../data/target/train/CED_train.csv').values
tst_target = pd.read_csv('../data/target/test/CED_test.csv').values
graph_path = '../nets/CED/CED_apr4.meta'
model_path = '../nets/CED/CED_apr4'
this_scaler = joblib.load('../nets/CED/scaler.pkl')
pca = joblib.load("../nets/CED/pca.pkl")
###

In [5]:
#split training and validation sets 
trn_X, val_X,trn_y, val_y = cross_validation.train_test_split(
    trn_descs, trn_target, test_size=0.1, random_state=RS)

#PCA
trn_X = pca.transform(this_scaler.transform(trn_X))
val_X = pca.transform(this_scaler.transform(val_X))
tst_X = pca.transform(this_scaler.transform(tst_descs))

In [6]:
pred_val = run_model(graph_path, model_path, val_X)
pred_tst = run_model(graph_path, model_path, tst_X)

In [12]:
np.savetxt("../results/pred.csv",pred_tst)

In [18]:
print r2_score(val_y,pred_val)
print np.mean(mre(val_y,pred_val))

0.521809925494
39.8644821756
